In [1]:
import sys
sys.path.insert(0, '/Users/pyro/github/HiveHelper_on_PySpark/hhop') 
# for running .ipynb files anywhere outside of a current dir using the module hhop

import hhop # custom module
from hhop import DFExtender, SchemaManager #main classes
from funs import read_table, write_table_through_view, union_all # useful functions

from functools import reduce
from importlib import reload
import pandas as pd

from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.sql.window import Window as W

import shutil, os, time # working with FS

22/12/18 18:51:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Возможно, пригодятся 
1. .config("spark.hadoop.hive.exec.dynamic.partition", "true") \
2. .config("spark.hadoop.hive.exec.dynamic.partition.mode", "nonstrict") \

## Synth table

In [2]:
df_src = spark.read.csv('./synth_data/table1.csv', header=True, sep=';')

In [3]:
df_src.show()

+-----+----+----+------+--------+----------+----------+
|index| pk1| pk2|  var1|    var2|   dt_part|group_part|
+-----+----+----+------+--------+----------+----------+
|    1|key1|   1|value1|value2_1|2022-12-15|    group1|
|    2|key1|   1|value1|value2_1|2022-12-16|    group2|
|    3|key1|   2|value1|value2_1|2022-12-16|    group3|
|    4|key2|   2|  null|value2_1|2022-12-17|    group1|
|    5|key2|   3|value1|value2_1|2022-12-18|    group2|
|    6|key2|   4|value1|    null|2022-12-19|    group3|
|    7|key2|null|value1|value2_1|2022-12-19|    group4|
|    8|null|   4|value1|value2_1|2022-12-20|    group3|
|    9|null|null|value1|value2_1|2022-12-20|    group7|
+-----+----+----+------+--------+----------+----------+



In [4]:
df_src.write.mode('overwrite').partitionBy('dt_part', 'group_part').saveAsTable('default.part_table_test1')
df_src.write.mode('overwrite').saveAsTable('default.nonpart_table_test1')

## Info about table

### Reading table from Hive

In [5]:
df_hive = read_table('default.part_table_test1', verbose=True)

root
 |-- index: string (nullable = true)
 |-- pk1: string (nullable = true)
 |-- pk2: string (nullable = true)
 |-- var1: string (nullable = true)
 |-- var2: string (nullable = true)
 |-- dt_part: string (nullable = true)
 |-- group_part: string (nullable = true)

location: file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/part_table_test1
partition columns: ['dt_part', 'group_part']


In [6]:
df = DFExtender(df_hive, pk=['pk1', 'pk2'], verbose=True)

### NULL checks

In [7]:
df_filled = df_hive.fillna({'pk1': 'default', 'pk2': 'default'})

In [8]:
df_filled.show()

+-----+-------+-------+------+--------+----------+----------+
|index|    pk1|    pk2|  var1|    var2|   dt_part|group_part|
+-----+-------+-------+------+--------+----------+----------+
|    1|   key1|      1|value1|value2_1|2022-12-15|    group1|
|    2|   key1|      1|value1|value2_1|2022-12-16|    group2|
|    3|   key1|      2|value1|value2_1|2022-12-16|    group3|
|    5|   key2|      3|value1|value2_1|2022-12-18|    group2|
|    7|   key2|default|value1|value2_1|2022-12-19|    group4|
|    8|default|      4|value1|value2_1|2022-12-20|    group3|
|    4|   key2|      2|  null|value2_1|2022-12-17|    group1|
|    9|default|default|value1|value2_1|2022-12-20|    group7|
|    6|   key2|      4|value1|    null|2022-12-19|    group3|
+-----+-------+-------+------+--------+----------+----------+



In [9]:
# reload(hhop)
# from hhop import DFExtender
k = DFExtender(df_filled, pk=['pk1', 'pk2'], verbose=True)
k.getInfo()

You can access DF with PK duplicates in attribute `.df_duplicates_pk`

Count all:                9
Unique PK count:          8
PK with duplicates:       1

Null values in columns - {'column': [count NULL, share NULL]}:
{'var1': [1, 0.1111], 'var2': [1, 0.1111]}


In [10]:
df_out = k.getDFWithNull(['var1', 'var2'])

In [11]:
df_out.show()

+-----+----+---+------+--------+----------+----------+---------+
|index| pk1|pk2|  var1|    var2|   dt_part|group_part|cnt_nulls|
+-----+----+---+------+--------+----------+----------+---------+
|    4|key2|  2|  null|value2_1|2022-12-17|    group1|        1|
|    6|key2|  4|value1|    null|2022-12-19|    group3|        1|
+-----+----+---+------+--------+----------+----------+---------+



### PK checks

In [12]:
df_check = DFExtender(df_hive, ['pk1', 'pk2'], verbose=True)
df_check.getInfo()

You can access DF with PK duplicates in attribute `.df_duplicates_pk`

Count all:                9
Unique PK count:          8
PK with duplicates:       1
PK column 'pk1' contains empty values, be careful!
PK column 'pk2' contains empty values, be careful!

Null values in columns - {'column': [count NULL, share NULL]}:
{'pk1': [2, 0.2222], 'pk2': [2, 0.2222], 'var1': [1, 0.1111], 'var2': [1, 0.1111]}


In [13]:
df_check.df_duplicates_pk.show()

+-----+----+---+------+--------+----------+----------+------+
|index| pk1|pk2|  var1|    var2|   dt_part|group_part|cnt_pk|
+-----+----+---+------+--------+----------+----------+------+
|    1|key1|  1|value1|value2_1|2022-12-15|    group1|     2|
|    2|key1|  1|value1|value2_1|2022-12-16|    group2|     2|
+-----+----+---+------+--------+----------+----------+------+



In [14]:
# should i calculate PK duplicates also like this?
df.groupBy(['pk1', 'pk2']).count().orderBy(col('count').desc()).show()

+----+----+-----+
| pk1| pk2|count|
+----+----+-----+
|key1|   1|    2|
|null|null|    1|
|key2|null|    1|
|key1|   2|    1|
|key2|   4|    1|
|key2|   2|    1|
|null|   4|    1|
|key2|   3|    1|
+----+----+-----+



## Comparing tables

In [15]:
def write_synth_sample(name):
    (
        spark.read.csv(f'./synth_data/{name}.csv', header=True, sep=';')
        .write.mode('overwrite')
        .partitionBy('dt_part', 'group_part')
        .saveAsTable(f'default.{name}')
    )
    
write_synth_sample('table1_comp')  
write_synth_sample('table2_comp')

df = read_table('default.table1_comp', alias='main')
df_ref = read_table('default.table2_comp', alias='ref')

In [16]:
# For testing DFs without common columns outside of PK
# df=df.select(['pk1', 'pk2'])
# df_ref=df_ref.select(['pk1', 'pk2'])

In [17]:
# reload(hhop)
# from hhop import DFExtender

df_main = DFExtender(df, pk=['pk1', 'pk2'], verbose=True)
df_main.compareTables(df_ref)

Main DF
Count all:                6
Unique PK count:          6
PK with duplicates:       0

Reference DF
Count all:                6
Unique PK count:          6
PK with duplicates:       0

Errors in columns - {'column': [count is_error, share is_error]}
{'group_part': [2, 0.4], 'var1': [2, 0.4], 'var2': [1, 0.2]}

Count stats of matching main and reference tables:
not in main table:        1
not in reference table:   1
correct matching:         5


In [18]:
df_main.df_with_errors.show()

+----+---+---------------+------------+----------+---------+---------+--------------+-----------+---------+--------------+---------+------------------+---------------+-------------+------------+------------+--------------+-------------+----------+
| pk1|pk2|group_part_main|dt_part_main|index_main|var1_main|var2_main|group_part_ref|dt_part_ref|index_ref|      var1_ref| var2_ref|group_part_is_diff|dt_part_is_diff|index_is_diff|var1_is_diff|var2_is_diff|is_joined_main|is_joined_ref|sum_errors|
+----+---+---------------+------------+----------+---------+---------+--------------+-----------+---------+--------------+---------+------------------+---------------+-------------+------------+------------+--------------+-------------+----------+
|key1|  1|         group2|  2022-12-15|         1|   value1| value2_1|        group7| 2022-12-15|        1|       value19| value2_1|                 1|              0|            0|           1|           0|             1|            1|         2|
|key2|  

## SchemaManager

### dropping empty tables

In [19]:
schema_name = 'popular_schema'
spark.sql(f"drop database if exists {schema_name} cascade")
spark.sql(f'create database {schema_name}')

DataFrame[]

In [20]:
df_src_write = df_src.write.mode('overwrite')
df_src_write.partitionBy('dt_part', 'group_part').saveAsTable(f'{schema_name}.table1')
df_src_write.saveAsTable(f'{schema_name}.table2')
df_src_write.saveAsTable(f'{schema_name}.table3')

table1 has dir + data   
table2 has only root dir    
table3 doesn't have any dir and data

In [21]:
shutil.rmtree('./spark-warehouse/popular_schema.db/table2', ignore_errors=True)
os.makedirs('./spark-warehouse/popular_schema.db/table2')
shutil.rmtree('./spark-warehouse/popular_schema.db/table3', ignore_errors=True)

In [22]:
spark.sql(f"show tables in {schema_name}").show(10, False)

+--------------+---------+-----------+
|database      |tableName|isTemporary|
+--------------+---------+-----------+
|popular_schema|table1   |false      |
|popular_schema|table2   |false      |
|popular_schema|table3   |false      |
+--------------+---------+-----------+



In [23]:
popular_schema = SchemaManager('popular_schema')

3 tables in popular_schema
run drop_empty_tables() on instance to drop empty tables in popular_schema


In [24]:
popular_schema.drop_empty_tables()

2 going to be dropped out of 3 (66.67%). Schema: popular_schema
After dropping tables there are 1 tables in popular_schema


22/12/18 18:52:09 ERROR FileUtils: Failed to delete file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/popular_schema.db/table3


In [25]:
spark.sql(f"show tables in {schema_name}").show(10, False)

+--------------+---------+-----------+
|database      |tableName|isTemporary|
+--------------+---------+-----------+
|popular_schema|table1   |false      |
+--------------+---------+-----------+



## Extra

### function `union_all`

In [26]:
# generating list of DataFrames
list_dfs = []
values = [
        ("x","x"),
        ("x","y"),
        ("x",None),
        (None,"x"),
        (None,None),
    ]
columns = ['val1', 'val2']
for val1, val2 in ((1,1), (1, None), (None, 1)):
    df_test = spark.createDataFrame(values, columns)
    df_test = (
        df_test
        .withColumn('is_joined_main', F.lit(val1))
        .withColumn('is_joined_ref', F.lit(val2))
    )
    list_dfs.append(df_test)

You can pass any number of Spark DataFrames is `union_all()`  
Requirements:
1. all DFs must have same columns
2. If `dfs` is a list, explode it like `*dfs` 

In [27]:
print(len(list_dfs))
list_dfs

3


[DataFrame[val1: string, val2: string, is_joined_main: int, is_joined_ref: int],
 DataFrame[val1: string, val2: string, is_joined_main: int, is_joined_ref: null],
 DataFrame[val1: string, val2: string, is_joined_main: null, is_joined_ref: int]]

In [28]:
df_from_union = union_all(*list_dfs).cache()
# union_all(list_dfs[0], list_dfs[1], list_dfs[2]) # equivalent
df_from_union

DataFrame[val1: string, val2: string, is_joined_main: int, is_joined_ref: int]

In [29]:
# this is exactly a filter in the script for comparing tables
dummy1, dummy2,val1,val2='is_joined_main','is_joined_ref','val1','val2'
cond_diff = f"""case
            when
                ({dummy1} is null or {dummy2} is null) 
                or
                ({val1} is null and {val2} is null)
                or 
                ({val1} = {val2})
                then 0
                else 1
            end"""
(
    df_from_union
    .withColumn('is_diff', F.expr(cond_diff))
    .show(100)
)

+----+----+--------------+-------------+-------+
|val1|val2|is_joined_main|is_joined_ref|is_diff|
+----+----+--------------+-------------+-------+
|   x|   x|             1|            1|      0|
|   x|   y|             1|            1|      1|
|   x|null|             1|            1|      1|
|null|   x|             1|            1|      1|
|null|null|             1|            1|      0|
|   x|   x|             1|         null|      0|
|   x|   y|             1|         null|      0|
|   x|null|             1|         null|      0|
|null|   x|             1|         null|      0|
|null|null|             1|         null|      0|
|   x|   x|          null|            1|      0|
|   x|   y|          null|            1|      0|
|   x|null|          null|            1|      0|
|null|   x|          null|            1|      0|
|null|null|          null|            1|      0|
+----+----+--------------+-------------+-------+



splitting big dataset if applicable

writing tables

## Modification

1. read as you like, use DFExtender to get stats
2. use all methods from PySpark as usual (but they convert DF from DFExtender to DataFrame)

In [30]:
df = read_table('default.table1_comp', alias='main')
df_main = DFExtender(df, pk=['pk1', 'pk2'], verbose=True)

In [31]:
print(df_main.__class__)

<class 'hhop.DFExtender'>


In [32]:
df_main_filter = df_main.filter(col('pk1').isNotNull())

In [33]:
print(df_main_filter.__class__)

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
spark.stop()